In [1]:
from utils.structure import *
from utils.predicter import *
from utils.solver import *

In [2]:
# build structure and display
test_datas = open('./data/test_demo.txt', 'r')
test_datas = test_datas.readlines()

In [3]:
# input test
for test_data in test_datas[:5]:
    s = Structure(test_data)
    print(s.data)
    print(s.display())

['1', '.', '.', '4', '3', '4', '1', '.', '.', '3', '2', '1', '2', '.', '.', '3']
+---+---+
|1 .|. 4|
|   |   |
|3 4|1 .|
+---+---+
|. 3|2 1|
|   |   |
|2 .|. 3|
+---+---+

['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '6', '.', '9', '3', '.', '.', '.', '9', '.', '.', '7', '6', '.', '.', '.', '4', '4', '.', '.', '.', '.', '6', '.', '3', '.', '.', '.', '.', '8', '.', '.', '.', '.', '2', '.', '1', '.', '.', '.', '.', '8', '5', '.', '7', '.', '.', '6', '5', '.', '.', '4', '.', '.', '8', '4', '.', '.', '.', '9', '.', '.', '.', '.', '3', '2', '.', '.', '.', '.', '.']
+-----+-----+-----+
|. . .|. . .|. . .|
|     |     |     |
|. . 6|. 9 3|. . .|
|     |     |     |
|9 . .|7 6 .|. . 4|
+-----+-----+-----+
|4 . .|. . 6|. 3 .|
|     |     |     |
|. . .|8 . .|. . 2|
|     |     |     |
|. 1 .|. . .|8 5 .|
+-----+-----+-----+
|7 . .|6 5 .|. 4 .|
|     |     |     |
|. 8 4|. . .|9 . .|
|     |     |     |
|. . 3|2 . .|. . .|
+-----+-----+-----+

['.', '5', '2', '.', '.', '9', '.', '.', 

In [4]:
# scan all area test
for i in range(6, 10):
    print(i, ':')
    s = Structure(test_datas[i])
    # print(s.display())
    solver = BasicSolver(s)
    print('- Before ', solver.scan_all(method='area', save_ready=True))
    print('- Processing ', solver.scan_all())
    print('- After ', solver.scan_all(method='area', save_ready=True))

6 :
- Before  False
- Processing  False
- After  True
7 :
- Before  True
- Processing  False
- After  True
8 :
- Before  True
- Processing  False
- After  True
9 :
- Before  False
- Processing  False
- After  True


In [5]:
# scan all group test
for i in range(11, 16):
    print(i, ':')
    s = Structure(test_datas[i])
    # print(s.display())
    solver = BasicSolver(s)
    print('- Before ', solver.scan_all(method='group', save_ready=True))
    print('- Processing ', solver.scan_all())
    print('- After ', solver.scan_all(method='group', save_ready=True))

11 :
Found the group drop part is ({0, 1, 2}, {0, 1, 2}, {0, 1, 2})  rows.
Found the group drop part is ({8, 6, 7}, {8, 6, 7}, {8, 6, 7})  rows.
Found the group drop part is ({0, 1, 2}, {0, 1, 2}, {0, 1, 2})  rows.
Found the group drop part is ({8, 6, 7}, {8, 6, 7}, {8, 6, 7})  rows.
Found the group drop part is ({0, 1, 2}, {0, 1, 2}, {0, 1, 2})  rows.
Found the group drop part is ({8, 6, 7}, {8, 6, 7}, {8, 6, 7})  rows.
Found the group drop part is ({0, 1, 2}, {0, 1, 2}, {0, 1, 2})  rows.
Found the group drop part is ({8, 6, 7}, {8, 6, 7}, {8, 6, 7})  rows.
Found the group drop part is ({0, 1, 2}, {0, 1, 2}, {0, 1, 2})  rows.
Found the group drop part is ({8, 6, 7}, {8, 6, 7}, {8, 6, 7})  rows.
Found the group drop part is ({0, 1, 2}, {0, 1, 2}, {0, 1, 2})  rows.
Found the group drop part is ({8, 6, 7}, {8, 6, 7}, {8, 6, 7})  rows.
Before grouped drop, scanned drop can also make some changes.
Found the group drop part is ({0, 1, 2}, {0, 1, 2}, {0, 1, 2})  rows.
Found the group drop pa

In [6]:
# scan all square test
for i in range(17, 22):
    print(i, ':')
    s = Structure(test_datas[i])
    # print(s.display())
    solver = BasicSolver(s)
    print('- Before ', solver.scan_all(method='square', save_ready=True))
    print('- Processing ', solver.scan_all())
    print('- After ', solver.scan_all(method='square', save_ready=True))

17 :
- Before  False
- Processing  False
Found the square drop part is ({0, 1}, {0, 1})  rows.
Found the square drop part is ({8, 5}, {8, 5})  cols.
Found the square drop part is ({8, 2}, {8, 2})  cols.
- After  True
18 :
- Before  False
- Processing  False
Found the square drop part is ({4, 7}, {4, 7})  rows.
Found the square drop part is ({0, 1}, {0, 1})  cols.
Found the square drop part is ({8, 4}, {8, 4})  rows.
Found the square drop part is ({0, 2}, {0, 2})  rows.
Found the square drop part is ({0, 4}, {0, 4})  cols.
- After  True
19 :
- Before  False
- Processing  False
Found the square drop part is ({3, 4}, {3, 4})  rows.
- After  True
20 :
- Before  False
- Processing  False
Found the square drop part is ({2, 4}, {2, 4})  rows.
Found the square drop part is ({3, 4}, {3, 4})  cols.
Found the square drop part is ({0, 2}, {0, 2})  rows.
Found the square drop part is ({2, 3}, {2, 3})  cols.
- After  True
21 :
- Before  False
- Processing  False
Found the square drop part is ({8, 7}

In [13]:
# check and step test
for test_data in test_datas:
    if test_data != '\n':
        s = Structure(test_data)
        solver = BasicSolver(s)
#         print(s.data)
        print('check : ', solver.check())
        solver = BasicSolver(s)
        print('step : ', solver.step())

Before area drop, scanned drop can also make some changes.
Before area drop, scanned drop can also make some changes.
Before area drop, scanned drop can also make some changes.
Before area drop, scanned drop can also make some changes.
check :  True
step :  True
Before area drop, scanned drop can also make some changes.
Before area drop, scanned drop can also make some changes.
Found the group drop part is ({3, 5}, {3, 5})  rows.
Found the group drop part is ({6, 7}, {6, 7})  rows.
Found the group drop part is ({4, 5}, {4, 5})  cols.
Found the group drop part is ({6, 7}, {6, 7})  cols.
Found the group drop part is ({0, 1, 2}, {0, 1, 2}, {0, 1, 2})  cols.
Found the group drop part is ({3, 4}, {3, 4})  rows.
Found the group drop part is ({8, 7}, {8, 7})  rows.
Found the group drop part is ({0, 1, 2}, {0, 1, 2}, {0, 1, 2})  rows.
Found the group drop part is ({3, 5}, {3, 5})  cols.
Found the group drop part is ({8, 6}, {8, 6})  cols.
Found the group drop part is ({0, 1}, {0, 1})  rows.
Fo

Before area drop, scanned drop can also make some changes.
Before area drop, scanned drop can also make some changes.
Before area drop, scanned drop can also make some changes.
Before area drop, scanned drop can also make some changes.
Before area drop, scanned drop can also make some changes.
Before grouped drop, scanned drop can also make some changes.
Found the group drop part is ({4, 5, 6}, {4, 5, 6}, {4, 5, 6})  rows.
Found the group drop part is ({8, 9, 10}, {8, 9, 10}, {8, 9, 10})  rows.
Found the group drop part is ({9, 10}, {9, 10})  cols.
Found the group drop part is ({4, 6}, {4, 6})  cols.
Before grouped drop, scanned drop can also make some changes.
Found the group drop part is ({13, 14}, {13, 14})  rows.
Found the group drop part is ({4, 6, 7}, {4, 6, 7}, {4, 6, 7})  cols.
Found the group drop part is ({2, 3}, {2, 3})  rows.
Found the group drop part is ({8, 9, 10, 11}, {8, 9, 10, 11}, {8, 9, 10, 11}, {8, 9, 10, 11})  rows.
Found the group drop part is ({1, 2}, {1, 2})  co

Found the group drop part is ({8, 9}, {8, 9})  rows.
Found the group drop part is ({8, 10, 11}, {8, 10, 11}, {8, 10, 11})  rows.
Found the group drop part is ({4, 5, 6, 7}, {4, 5, 6, 7}, {4, 5, 6, 7}, {4, 5, 6, 7})  rows.
Found the group drop part is ({2, 3}, {2, 3})  cols.
Found the group drop part is ({4, 7}, {4, 7})  cols.
Found the group drop part is ({9, 10}, {9, 10})  cols.
Found the group drop part is ({2, 3}, {2, 3})  rows.
Found the group drop part is ({5, 7}, {5, 7})  rows.
Found the group drop part is ({2, 3}, {2, 3})  cols.
Found the group drop part is ({12, 13}, {12, 13})  cols.
Found the group drop part is ({8, 11}, {8, 11})  rows.
Found the group drop part is ({1, 2}, {1, 2})  cols.
Found the square drop part is ({12, 13}, {12, 13})  rows.
Found the square drop part is ({12, 13}, {12, 13})  rows.
Found the square drop part is ({12, 13}, {12, 13})  rows.
Found the square drop part is ({12, 13}, {12, 13})  rows.
Found the square drop part is ({12, 13}, {12, 13})  rows.
Fou

Found the group drop part is ({1, 2}, {1, 2})  rows.
Found the group drop part is ({6, 7}, {6, 7})  rows.
Found the group drop part is ({6, 7}, {6, 7})  cols.
Found the group drop part is ({0, 1}, {0, 1})  rows.
Found the group drop part is ({6, 7}, {6, 7})  cols.
Found the group drop part is ({4, 5}, {4, 5})  cols.
Found the group drop part is ({0, 1}, {0, 1})  rows.
Found the group drop part is ({1, 2}, {1, 2})  cols.
Found the group drop part is ({0, 1, 2}, {0, 1, 2}, {0, 1, 2})  rows.
Found the group drop part is ({1, 2}, {1, 2})  cols.
step :  True
Before area drop, scanned drop can also make some changes.
Before area drop, scanned drop can also make some changes.
Before area drop, scanned drop can also make some changes.
Before grouped drop, scanned drop can also make some changes.
Found the group drop part is ({3, 5}, {3, 5})  rows.
Found the group drop part is ({0, 1}, {0, 1})  cols.
Before grouped drop, scanned drop can also make some changes.
Before grouped drop, scanned drop

Before grouped drop, scanned drop can also make some changes.
Found the group drop part is ({0, 2}, {0, 2})  rows.
Found the group drop part is ({3, 5}, {3, 5})  rows.
Found the group drop part is ({1, 2}, {1, 2})  cols.
Before grouped drop, scanned drop can also make some changes.
Found the group drop part is ({0, 2}, {0, 2})  rows.
Found the group drop part is ({8, 7}, {8, 7})  rows.
Found the group drop part is ({6, 7}, {6, 7})  cols.
Found the group drop part is ({1, 2}, {1, 2})  cols.
Before grouped drop, scanned drop can also make some changes.
Before grouped drop, scanned drop can also make some changes.
Found the group drop part is ({8, 7}, {8, 7})  rows.
Found the group drop part is ({0, 1}, {0, 1})  cols.
Found the group drop part is ({8, 6}, {8, 6})  cols.
Before grouped drop, scanned drop can also make some changes.
Found the group drop part is ({0, 2}, {0, 2})  rows.
Found the group drop part is ({4, 5}, {4, 5})  cols.
Found the square drop part is ({0, 1}, {0, 1})  rows.


In [8]:
solver.steps

[(70, '1'), (79, '1'), (1, '3'), (9, '3'), (11, '3')]

In [9]:
tmp = "	 	 	6	 	 	4	1	7	2\
	 	 	 	2	 	1	5	 	 \
	 	1	 	 	3	 	9	 	 \
	 	2	8	 	 	 	3	 	4\
	1	 	 	 	 	 	2	 	7\
	6	 	 	 	 	 	8	1	 \
	 	 	3	 	7	 	4	 	 \
	 	 	1	3	 	5	7	 	 \
	7	9	 	4	 	 	6	 	 "

In [10]:
tmp = tmp.replace(' ', '.').replace('×', '.').replace('?','.').replace('	', '')
print(len(','.join(list(tmp))))
','.join(list(tmp))

161


'.,.,6,.,.,4,1,7,2,.,.,.,2,.,1,5,.,.,.,1,.,.,3,.,9,.,.,.,2,8,.,.,.,3,.,4,1,.,.,.,.,.,2,.,7,6,.,.,.,.,.,8,1,.,.,.,3,.,7,.,4,.,.,.,.,1,3,.,5,7,.,.,7,9,.,4,.,.,6,.,.'